In [1]:
!pip install dtaidistance

     |████████████████████████████████| 797 kB 26.3 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dtaidistance: filename=dtaidistance-2.3.6-cp36-cp36m-linux_x86_64.whl size=892044 sha256=876f6f0a64eb337eeff24b7ada37c94acd09480d385c3534206b9da38219045d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/30/5f/18/3db8e0572e06cde30543380d60b4511e3802d1481c3a3a2642
Successfully built dtaidistance


In [2]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from collections import defaultdict
from tqdm import tqdm 
from dtaidistance import dtw
from heapq import nlargest

tqdm.pandas()

In [3]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [4]:
df_pred.head()

,candidate_id,job_order,time_between,time_spent,isco_functie_niveau,source,education,company_name,function_id,isco_code4
0,84556,0,37,156,2.0,0,0.0,324258,936,208
1,84556,1,23,116,1.0,0,0.0,324258,809,348
2,84556,2,23,275,2.0,0,0.0,329244,936,208
3,84556,3,1155,4,1.0,0,0.0,368140,1519,344
4,84556,4,203,11,1.0,0,0.0,419826,1519,344


In [5]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [6]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:17<00:00, 27307.54it/s]


In [7]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [8]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]

In [ ]:
results = defaultdict(lambda: defaultdict(int))

try:
    for i, current in tqdm(enumerate(career_paths.iteritems()), total=len(career_paths)):
        candidate, career = current
        career = career[:-1].astype("double")

        for j in career_paths.iteritems():
            candidate2, career2 = j

            if candidate != candidate2:
                results[candidate][candidate2] = dtw.distance(career, 
                                                              career2.astype("double"), 
                                                              use_c=True, 
                                                              use_pruning=True)
                
        if i == 1500:
            break
                        
except KeyboardInterrupt:
    pass

  1%|▏         | 1500/113724 [32:18<40:17:30,  1.29s/it]


In [ ]:
best_matches = {}

for i in tqdm(results):
    if len(results[i]):
        best_matches[i] = sorted(results[i], key=lambda x: results[i][x])

100%|██████████| 1501/1501 [01:43<00:00, 14.57it/s]


In [ ]:
def knn_classifier(best_matches, career_paths, k=1, n=1):
    
    predictions = []
    
    for candidate in best_matches.keys():
        correct = career_paths[candidate][-1]
                
        neighbors = [(career_paths[neighbor][-1], i + 1) for i, neighbor in enumerate(best_matches[candidate][:k])]
        
        weights = defaultdict(int)
        
        for job, weight in neighbors:
            weights[job] += 1/np.log(1 + weight)
            
        predictions.append(correct in nlargest(n, weights, key=weights.get))

    
    return np.mean(predictions)

In [17]:
results = defaultdict(list)

for k in tqdm(range(1, 20)):
    results["k"].append(k)
    for n in [1, 5, 10, 20]:
        results[f"accuracy@{n}"].append(knn_classifier(best_matches, career_paths, k=k, n=n))
    
pd.DataFrame(results).set_index("k")

100%|██████████| 19/19 [00:12<00:00,  1.47it/s]


,accuracy@1,accuracy@5,accuracy@10,accuracy@20
k,,,,
1,0.063291,0.063291,0.063291,0.063291
2,0.063291,0.112592,0.112592,0.112592
3,0.067288,0.144570,0.144570,0.144570
4,0.065956,0.169887,0.169887,0.169887
5,0.071952,0.192538,0.192538,0.192538
6,0.073284,0.204530,0.204530,0.204530
7,0.075283,0.215856,0.219187,0.219187
8,0.076616,0.224517,0.229847,0.229847
9,0.079280,0.231845,0.239840,0.239840
